In [2]:
import numpy as np
import pandas as pd
import indicators as ind #indicators.pyのインポート

dataM1 = pd.read_csv('DAT_ASCII_EURUSD_M1_2015.csv', sep=';',
                     names=('Time','Open','High','Low','Close', ''),
                     index_col='Time', parse_dates=True)
dataM1.index += pd.offsets.Hour(7) #7時間のオフセット
ohlc = ind.TF_ohlc(dataM1, 'H') #１時間足データの作成

In [3]:
FastMA = ind.iMA(ohlc, 10) #エントリー用短期移動平均
SlowMA = ind.iMA(ohlc, 30) #エントリー用長期移動平均
ExitMA = ind.iMA(ohlc, 5) #エグジット用移動平均

In [4]:
#買いエントリーシグナル
BuyEntry = ((FastMA > SlowMA) & (FastMA.shift() <= SlowMA.shift())).values
#売りエントリーシグナル
SellEntry = ((FastMA < SlowMA) & (FastMA.shift() >= SlowMA.shift())).values
#買いエグジットシグナル
BuyExit = ((ohlc.Close < ExitMA) & (ohlc.Close.shift() >= ExitMA.shift())).values
#売りエグジットシグナル
SellExit = ((ohlc.Close > ExitMA) & (ohlc.Close.shift() <= ExitMA.shift())).values

In [5]:
def Backtest(ohlc, BuyEntry, SellEntry, BuyExit, SellExit, lots=0.1, spread=2, TP=0, SL=0, Limit=0, Expiration=10):
    Open = ohlc.Open.values #始値
    Low = ohlc.Low.values #安値
    High = ohlc.High.values #高値
    Point = 0.0001 #1pipの値
    if(Open[0] > 50): Point = 0.01 #クロス円の1pipの値
    Spread = spread*Point #スプレッド
    Lots = lots*100000 #実際の売買量
    N = len(ohlc) #FXデータのサイズ

    LongTrade = np.zeros(N) #買いトレード情報
    ShortTrade = np.zeros(N) #売りトレード情報

    #買いエントリー価格
    BuyEntryS = np.hstack((False, BuyEntry[:-1])) #買いエントリーシグナルのシフト
    if Limit == 0: LongTrade[BuyEntryS] = Open[BuyEntryS]+Spread #成行買い
    else: #指値買い
        for i in range(N-Expiration):
            if BuyEntryS[i]:
                BuyLimit = Open[i]-Limit*Point #指値
                for j in range(Expiration):
                    if Low[i+j] <= BuyLimit: #約定条件
                        LongTrade[i+j] = BuyLimit+Spread
                        break
    #買いエグジット価格
    BuyExitS = np.hstack((False, BuyExit[:-2], True)) #買いエグジットシグナルのシフト
    LongTrade[BuyExitS] = -Open[BuyExitS]

    #売りエントリー価格
    SellEntryS = np.hstack((False, SellEntry[:-1])) #売りエントリーシグナルのシフト
    if Limit == 0: ShortTrade[SellEntryS] = Open[SellEntryS] #成行売り
    else: #指値売り
        for i in range(N-Expiration):
            if SellEntryS[i]:
                SellLimit = Open[i]+Limit*Point #指値
                for j in range(Expiration):
                    if High[i+j] >= SellLimit: #約定条件
                        ShortTrade[i+j] = SellLimit
                        break
    #売りエグジット価格
    SellExitS = np.hstack((False, SellExit[:-2], True)) #売りエグジットシグナルのシフト
    ShortTrade[SellExitS] = -(Open[SellExitS]+Spread)

    LongPL = np.zeros(N) # 買いポジションの損益
    ShortPL = np.zeros(N) # 売りポジションの損益
    BuyPrice = SellPrice = 0.0 # 売買価格

    for i in range(1,N):
        if LongTrade[i] > 0: #買いエントリーシグナル
            if BuyPrice == 0:
                BuyPrice = LongTrade[i]
                ShortTrade[i] = -BuyPrice #売りエグジット
            else: LongTrade[i] = 0

        if ShortTrade[i] > 0: #売りエントリーシグナル
            if SellPrice == 0:
                SellPrice = ShortTrade[i]
                LongTrade[i] = -SellPrice #買いエグジット
            else: ShortTrade[i] = 0

        if LongTrade[i] < 0: #買いエグジットシグナル
            if BuyPrice != 0:
                LongPL[i] = -(BuyPrice+LongTrade[i])*Lots #損益確定
                BuyPrice = 0
            else: LongTrade[i] = 0

        if ShortTrade[i] < 0: #売りエグジットシグナル
            if SellPrice != 0:
                ShortPL[i] = (SellPrice+ShortTrade[i])*Lots #損益確定
                SellPrice = 0
            else: ShortTrade[i] = 0

        if BuyPrice != 0 and SL > 0: #SLによる買いポジションの決済
            StopPrice = BuyPrice-SL*Point
            if Low[i] <= StopPrice:
                LongTrade[i] = -StopPrice
                LongPL[i] = -(BuyPrice+LongTrade[i])*Lots #損益確定
                BuyPrice = 0

        if BuyPrice != 0 and TP > 0: #TPによる買いポジションの決済
            LimitPrice = BuyPrice+TP*Point
            if High[i] >= LimitPrice:
                LongTrade[i] = -LimitPrice
                LongPL[i] = -(BuyPrice+LongTrade[i])*Lots #損益確定
                BuyPrice = 0

        if SellPrice != 0 and SL > 0: #SLによる売りポジションの決済
            StopPrice = SellPrice+SL*Point
            if High[i] >= StopPrice+Spread:
                ShortTrade[i] = -StopPrice
                ShortPL[i] = (SellPrice+ShortTrade[i])*Lots #損益確定
                SellPrice = 0

        if SellPrice != 0 and TP > 0: #TPによる売りポジションの決済
            LimitPrice = SellPrice-TP*Point
            if Low[i] <= LimitPrice+Spread:
                ShortTrade[i] = -LimitPrice
                ShortPL[i] = (SellPrice+ShortTrade[i])*Lots #損益確定
                SellPrice = 0

    return pd.DataFrame({'Long':LongTrade, 'Short':ShortTrade}, index=ohlc.index),\
            pd.DataFrame({'Long':LongPL, 'Short':ShortPL}, index=ohlc.index)

In [6]:
Trade, PL = Backtest(ohlc, BuyEntry, SellEntry, BuyExit, SellExit, TP=100, SL=50, Limit=20)

In [8]:
from pandas_highcharts.display import display_charts
Initial = 10000 # 初期資産
Equity = (PL.Long+PL.Short).cumsum() #累積損益
display_charts(pd.DataFrame({'Equity':Equity+Initial}), chart_type="stock", title="資産曲線", figsize=(640,480), grid=True)